In [ ]:
    #Define the constraints
    def slip_rule(model, i, t):
        return model.u_wheels[i,t] - (model.u[i,t] + model.u_added_slip[i,t]) == 0 if t > 0 and t<model.N-1 else pyo.Constraint.Skip
    def slip_rule2(model, i, t):
        return model.u_slip[i,t] - (model.u[i,t]/((model.u_diff[i,t])**2+1)) == 0 if t > 0 and t<model.N-1 else pyo.Constraint.Skip

    def sys_dyn0_rule(model,t):
        for t in range(model.N-1):
            if model.u_added_slip[0,t] <= 0.1:
                return model.q[0, t+1] == model.q[0, t] + Ts*(r/2)*pyo.cos(model.q[2,t]) * (model.u[0, t] + model.u[1, t])
            else:
                return model.q[0, t+1] == model.q[0, t]
        return pyo.Constraint.Skip
    def sys_dyn1_rule(model,t):
        if t<model.N:
            if model.u_added_slip[1,t] <= 0.1:
                return model.q[1, t+1] == model.q[1, t] + Ts*(r/2)*pyo.sin(model.q[2,t]) * (model.u[0, t] + model.u[1, t])
            else:
                return model.q[1, t+1] == model.q[1, t]
        return pyo.Constraint.Skip
    def sys_dyn2_rule(model,t):
        if t<model.N:
            if model.u_added_slip[0,t] <= 0.1 and model.u_added_slip[1,t] <= 0.1:
                return model.q[2, t+1] == model.q[2, t] + Ts*(r/L)*(-model.u[0, t] + model.u[1, t])
            else:
                return model.q[2, t+1] == model.q[2, t]
        return pyo.Constraint.Skip
    

In [ ]:
# GLOBAL VARIABLES
r_cm = 3   # cm 
L_cm = 5  # cm
Ts = 0.1   # s
MAX_U = 10 # rad/s
MAX_deltaU = 10

def solve_cftoc_batch(q0, qf, N, P, r, L, Ts, MAX_deltaU):
    model = pyo.ConcreteModel()

    #Define the variables
    nq = 5
    nu = 2
    model.N = N
    model.tidx = pyo.Set(initialize=range(N+1), ordered=True)
    model.tuidx = pyo.Set(initialize=range(N), ordered=True)
    model.qidx = pyo.Set(initialize=range(nq), ordered=True)
    model.uidx = pyo.Set(initialize=range(nu), ordered=True)

    model.q = pyo.Var(model.qidx, model.tidx)
    model.u = pyo.Var(model.uidx, model.tuidx, bounds=(-MAX_U/2, MAX_U/2))
    model.u_slip = pyo.Var(model.uidx, model.tuidx, bounds=(-MAX_U, MAX_U))    

    
    model.u_added_slip = pyo.Expression(model.uidx, model.tuidx, rule=lambda model, i, t: ((model.u[i, t + 1] - model.u[i, t - 1]) / 4)**7 if 0 < t < model.N - 1 else 0)

    #Define the constraints
    def slip_rule(model, i, t):
        return model.u_slip[i,t] - (model.u[i,t] + model.u_added_slip[i,t]) == 0 if t > 0 and t<model.N-1 else pyo.Constraint.Skip

    def sys_dyn0_rule(model,t):
        for t in range(model.N-1):
            if model.u_added_slip[0,t] <= 0.1:
                return model.q[0, t+1] == model.q[0, t] + Ts*(r/2)*pyo.cos(model.q[2,t]) * (model.u[0, t] + model.u[1, t])
            else:
                return model.q[0, t+1] == model.q[0, t]
        return pyo.Constraint.Skip
    def sys_dyn1_rule(model,t):
        if t<model.N:
            if model.u_added_slip[1,t] <= 0.1:
                return model.q[1, t+1] == model.q[1, t] + Ts*(r/2)*pyo.sin(model.q[2,t]) * (model.u[0, t] + model.u[1, t])
            else:
                return model.q[1, t+1] == model.q[1, t]
        return pyo.Constraint.Skip
    def sys_dyn2_rule(model,t):
        if t<model.N:
            if model.u_added_slip[0,t] <= 0.1 and model.u_added_slip[1,t] <= 0.1:
                return model.q[2, t+1] == model.q[2, t] + Ts*(r/L)*(-model.u[0, t] + model.u[1, t])
            else:
                return model.q[2, t+1] == model.q[2, t]
        return pyo.Constraint.Skip
    
    #initial condition - fix these at the initial state
    model.init_condition = pyo.Constraint(model.qidx, rule=lambda model, i: model.q[i,0] == q0[i])
    
    #System dynamics, don't include the last time step, so use tuidx
    model.sys_dyn0 = pyo.Constraint(model.tidx, rule=sys_dyn0_rule)
    model.sys_dyn1 = pyo.Constraint(model.tidx, rule=sys_dyn1_rule)
    model.sys_dyn2 = pyo.Constraint(model.tidx, rule=sys_dyn2_rule)
    model.sys_dyn3 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.q[3, t+1] == model.q[3, t] + Ts*model.u_slip[0, t])
    model.sys_dyn4 = pyo.Constraint(model.tuidx, rule=lambda model, t: model.q[4, t+1] == model.q[4, t] + Ts*model.u_slip[1, t])
    
    #Slip Dynamics
    model.sys_dyn5 = pyo.Constraint(model.uidx, model.tuidx, rule=slip_rule)
    model.sys_dyn6 = pyo.Constraint(expr = model.u_slip[0,0] == model.u[0,0])
    model.sys_dyn7 = pyo.Constraint(expr = model.u_slip[1,0] == model.u[1,0])
    model.sys_dyn8 = pyo.Constraint(expr = model.u_slip[0,N-1] == model.u[0,N-1])
    model.sys_dyn9 = pyo.Constraint(expr = model.u_slip[1,N-1] == model.u[1,N-1])
    
    
    #Final Constraint
    epsilon = 0.01
    # model.final_constraint0 = pyo.Constraint(expr = (model.q[0,N] - qf[0])**2 <= epsilon)
    model.final_constraint1 = pyo.Constraint(expr = (model.q[1,N] - qf[1])**2 <= epsilon)
    model.final_constraint2a = pyo.Constraint(expr = (pyo.cos(model.q[2,N]) - pyo.cos(qf[2]))**2 <= epsilon)
    model.final_constraint2b = pyo.Constraint(expr = (pyo.sin(model.q[2,N]) - pyo.sin(qf[2]))**2 <= epsilon)
    model.final_constraint3a = pyo.Constraint(expr = (pyo.cos(model.q[3,N]) - pyo.cos(qf[3]))**2 <= epsilon)
    model.final_constraint3b = pyo.Constraint(expr = (pyo.sin(model.q[3,N]) - pyo.sin(qf[3]))**2 <= epsilon)
    model.final_constraint4a = pyo.Constraint(expr = (pyo.cos(model.q[4,N]) - pyo.cos(qf[4]))**2 <= epsilon)
    model.final_constraint4b = pyo.Constraint(expr = (pyo.sin(model.q[4,N]) - pyo.sin(qf[4]))**2 <= epsilon)

    #Input constraint, max ΔU
    model.input_deltaConstraint0 = pyo.Constraint(model.tuidx, rule=lambda model, t: (model.u_slip[0,t+1] - model.u_slip[0,t])**2 <= MAX_deltaU if t < N-1 else pyo.Constraint.Skip)
    model.input_deltaConstraint1 = pyo.Constraint(model.tuidx, rule=lambda model, t: (model.u_slip[1,t+1] - model.u_slip[1,t])**2 <= MAX_deltaU if t < N-1 else pyo.Constraint.Skip)

    # Cost function
    def cost_rule(model):
        cost_q = 0
        cost_u = 0

        # Make the difference in wheel angle, heading for the last 5% of the time steps as small as possible
        for t in range(model.N):
            for i in model.qidx:
                cost_q += P[i,i] * (model.q[i,t] - qf[i])**2

        # Minimize the control effort
        for t in model.tuidx:
            cost_u += 2*model.u[0,t]**2 + 2*model.u[1,t]**2 # R

        return cost_q + cost_u
    
    model.cost = pyo.Objective(rule=cost_rule, sense=pyo.minimize)

    # Solve the optimization problem
    solver = pyo.SolverFactory('ipopt')
    results = solver.solve(model)
  
    feas = str(results.solver.termination_condition) == "optimal"
    qOpt = np.asarray([model.q[:,t]() for t in model.tidx]).T
    uOpt = np.asarray([model.u[:,t]() for t in model.tuidx]).T
    uslipOpt = np.asarray([model.u_slip[:,t]() for t in model.tuidx]).T
    JOpt = model.cost()
    # log_infeasible_constraints(model)

    return feas, qOpt, uOpt, uslipOpt, JOpt, model, results


q0 = np.array([0, 0, 0, 0, np.pi/2])
qf = np.array([0, 0, 0, 0, 0])
N = 100
P = np.diag([1, 1, 2, 2, 2])
feas, qOpt, uOpt, uslipOpt, JOpt, model, results = solve_cftoc_batch(q0, qf, N, P, r_cm, L_cm, Ts, MAX_deltaU)